In [5]:
from faster_whisper import WhisperModel
from typing import TypedDict, Annotated
from langchain_core.messages import BaseMessage
import openai
import os
from dotenv import load_dotenv
import re
import mysql.connector
from datetime import datetime

# 1️⃣ State 정의
class CallingState(TypedDict):
    full_text: Annotated[str, "Transcribed Text"]
    summary: Annotated[str, "Summarized Info"]
    messages: Annotated[list[BaseMessage], "Messages"]

# 2️⃣ 빈값 -> None 변환 함수
def empty_to_none(value):
    if value and value.lower() != "nan":
        return value.strip()
    else:
        return None

# 3️⃣ 음성 인식 → State 반환
def transcribe(state: CallingState) -> CallingState:
    model = WhisperModel("base")
    segments, info = model.transcribe("calling_data.m4a")
    full_text = " ".join([seg.text for seg in segments])
    print("📝 인식된 텍스트:")
    print(full_text)

    return CallingState(
        full_text=full_text,
        summary="",
        messages=[]
    )

# 4️⃣ 요약 및 요구사항 정리 → State 갱신
def summarize(state: CallingState) -> CallingState:
    load_dotenv()
    client = openai.OpenAI()

    prompt = f"""
    다음 고객과의 통화 내용을 요약하고, 고객의 요구사항을 다음 필드별로 정리해줘:
    - 브랜드 담당자 이름:
    - 연락처:
    - 지역:
    - 매체:
    - 타겟층:
    - 비고 (미팅 날짜, 추후 연락 등):
    - 성사 여부:

    통화 내용:
    \"\"\"{state['full_text']}\"\"\"
    """

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "너는 영업 담당자의 어시스턴트야. 고객의 요구사항을 정확하게 정리해야 해."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2
    )

    summary = response.choices[0].message.content
    print("🔎 요약 및 요구사항 정리:")
    print(summary)

    return CallingState(
        full_text=state["full_text"],
        summary=summary,
        messages=[]
    )

# 5️⃣ GPT 요약에서 필드별 값 추출
def extract_fields(summary: str, full_text: str):
    def extract(label):
        match = re.search(f"{label}: *(.*)", summary)
        return match.group(1).strip() if match else ""

    # 필드별 추출
    manager_name = extract("브랜드 담당자 이름")
    manager_email = extract("연락처")

    region = extract("지역")
    media = extract("매체")
    target = extract("타겟층")

    # call_memo는 **요약 전체**를 그대로 사용!
    call_memo = summary

    # client_needs_summary = 지역 + 매체 + 타겟층 이어 붙이기
    summary_parts = []
    if region:
        summary_parts.append(f"지역: {region}")
    if media:
        summary_parts.append(f"매체: {media}")
    if target:
        summary_parts.append(f"타겟층: {target}")
    client_needs_summary = "\n".join(summary_parts) if summary_parts else None

    return {
        "manager_name": manager_name,
        "manager_email": manager_email,
        "call_full_text": full_text,
        "call_memo": call_memo,  # 요약 전체가 메모로
        "client_needs_summary": client_needs_summary
    }

# 6️⃣ MariaDB에 데이터 저장
def save_to_mariadb(fields: dict):
    load_dotenv()
    conn = mysql.connector.connect(
        host=os.getenv("DB_HOST"),
        user=os.getenv("DB_USER"),
        password=os.getenv("DB_PASSWORD"),
        database=os.getenv("DB_NAME")
    )
    cursor = conn.cursor()

    sql = """
    INSERT INTO sales_log 
    (manager_name, manager_email, call_full_text, call_memo, client_needs_summary)
    VALUES (%s, %s, %s, %s, %s)
    """
    values = (
        empty_to_none(fields["manager_name"]),
        empty_to_none(fields["manager_email"]),
        empty_to_none(fields["call_full_text"]),
        empty_to_none(fields["call_memo"]),
        empty_to_none(fields["client_needs_summary"])
    )
    cursor.execute(sql, values)
    conn.commit()
    print("✅ sales_log 테이블에 데이터 저장 완료")
    cursor.close()
    conn.close()

# 7️⃣ 전체 파이프라인 실행
state: CallingState = {"full_text": "", "summary": "", "messages": []}

state = transcribe(state)
state = summarize(state)
fields = extract_fields(state["summary"], state["full_text"])
save_to_mariadb(fields)

print("🗂️ 최종 저장된 필드:")
print(fields)

📝 인식된 텍스트:
 안녕하세요 오개광고대 행사 올리질 굿입니다. 혹시 언어부 담당자 맞으신가요? 네 맞습니다.  네 안녕하세요. 다름이 아니라 이번 신제품 출시에 맞춰서 저희가 홍대 지하철 매체를 제한드리고자 하는데 혹시 관심 있으실까요? 홍대가 언어부 주요 타계층인 mg세대와 딱 맞을 것 같아서요.  아 저희가 이번에 홍대는 말고 성수의 파버스토리 계획인데 혹시 성숙 좀 매체도 있나요? 네 물론이죠. 성수역 근처 뿐만 아니라 성수 카페 거리 중심으로 저희가 여러 매체를 보유하고 있는데 혹시 제가 미팅으로 자세하게 설명드려도 될까요? 죄송한데 제가 미팅 어릴 것 같고요. 이멜로 일단 제한서 보내주세요. 네 그럼 저희 통합 오개 광고 제한서와 함께 성수 파벅을 맞춘 제한서 따로 보내드리겠습니다.  혹시 담담자분 성함과 이메일 알 수 있을까요? 제 이름은 이 효정이고요. 이메일은 sgsgk113 골뱅이 스크라다컴입니다. 네 sgsgk113 골뱅이 skal8.co에 맞으시죠? 오늘 중으로 매일 드리겠습니다. 감사합니다. 네 감사합니다.
🔎 요약 및 요구사항 정리:
- 브랜드 담당자 이름: 이효정
- 연락처: sgsgk113@skal8.co
- 지역: 성수
- 매체: 성수역 근처 및 성수 카페 거리 중심
- 타겟층: MG세대
- 비고 (미팅 날짜, 추후 연락 등): 미팅 대신 이메일로 제안서 요청, 오늘 중으로 이메일 발송 예정
- 성사 여부: 미정 (제안서 발송 후 결정될 가능성 있음)
✅ sales_log 테이블에 데이터 저장 완료
🗂️ 최종 저장된 필드:
{'manager_name': '이효정', 'manager_email': 'sgsgk113@skal8.co', 'call_full_text': ' 안녕하세요 오개광고대 행사 올리질 굿입니다. 혹시 언어부 담당자 맞으신가요? 네 맞습니다.  네 안녕하세요. 다름이 아니라 이번 신제품 출시에 맞춰서 저희가 홍대 지하철 매체를 제한드리고자 하는데 혹시 관심 있으실까요? 홍대가 언어부 주요 타계층인 mg세대와 딱 맞

In [ ]:
from faster_whisper import WhisperModel
from typing import TypedDict, Annotated
from langchain_core.messages import BaseMessage
import openai
import os
from dotenv import load_dotenv

# 1️⃣ State 정의
class CallingState(TypedDict):
    full_text: Annotated[str, "Transcribed Text"]
    summary: Annotated[str, "Summarized Info"]
    messages: Annotated[list[BaseMessage], "Messages"]  # 향후 확장용

# 2️⃣ 음성 인식 함수 → State 반환
def transcribe(state: CallingState) -> CallingState:
    model = WhisperModel("base")
    segments, info = model.transcribe("calling_data.m4a")
    full_text = " ".join([seg.text for seg in segments])
    print("📝 인식된 텍스트:")
    print(full_text)

    return CallingState(
        full_text=full_text,
        summary="",
        messages=[]
    )

# 3️⃣ 요약 및 요구사항 정리 함수 → State 갱신
def summarize(state: CallingState) -> CallingState:
    load_dotenv()
    client = openai.OpenAI()

    prompt = f"""
    다음 고객과의 통화 내용을 요약하고, 고객의 요구사항을 다음 필드별로 정리해줘:
    - 지역:
    - 매체:
    - 타겟층:
    - 비고 (미팅 날짜, 추후 연락 등):
    - 연락처:
    - 성사 여부:

    통화 내용:
    \"\"\"{state['full_text']}\"\"\" 
    """

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "너는 영업 담당자의 어시스턴트야. 고객의 요구사항을 정확하게 정리해야 해."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2
    )

    summary = response.choices[0].message.content
    print("🔎 요약 및 요구사항 정리:")
    print(summary)

    return CallingState(
        full_text=state["full_text"],
        summary=summary,
        messages=[]  # 나중에 GPT 메시지 기록 넣으면 됨
    )

# 4️⃣ 전체 파이프라인 실행
state: CallingState = {"full_text": "", "summary": "", "messages": []}

state = transcribe(state)
state = summarize(state)

print("🗂️ 최종 State:")
print(state)

# 이름, 이메일, 내용, 요구사항(지역, 매체, 타겟층), 메모(비고, 성사여부)

c:\Users\Administrator\anaconda3\envs\OoHMarketingSales\Lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


📝 인식된 텍스트:
 안녕하세요 오개광고대 행사 올리질 굿입니다. 혹시 언어부 담당자 맞으신가요? 네 맞습니다.  네 안녕하세요. 다름이 아니라 이번 신제품 출시에 맞춰서 저희가 홍대 지하철 매체를 제한드리고자 하는데 혹시 관심 있으실까요? 홍대가 언어부 주요 타계층인 mg세대와 딱 맞을 것 같아서요.  아 저희가 이번에 홍대는 말고 성수의 파버스토리 계획인데 혹시 성숙 좀 매체도 있나요? 네 물론이죠. 성수역 근처 뿐만 아니라 성수 카페 거리 중심으로 저희가 여러 매체를 보유하고 있는데 혹시 제가 미팅으로 자세하게 설명드려도 될까요? 죄송한데 제가 미팅 어릴 것 같고요. 이멜로 일단 제한서 보내주세요. 네 그럼 저희 통합 오개 광고 제한서와 함께 성수 파벅을 맞춘 제한서 따로 보내드리겠습니다.  혹시 담담자분 성함과 이메일 알 수 있을까요? 제 이름은 이 효정이고요. 이메일은 sgsgk113 골뱅이 스크라다컴입니다. 네 sgsgk113 골뱅이 skal8.co에 맞으시죠? 오늘 중으로 매일 드리겠습니다. 감사합니다. 네 감사합니다.
🔎 요약 및 요구사항 정리:
- 지역: 성수
- 매체: 성수역 근처 및 성수 카페 거리 중심의 매체
- 타겟층: MZ세대
- 비고 (미팅 날짜, 추후 연락 등): 미팅은 어려우며, 이메일로 제안서 요청
- 연락처: 이메일 - sgsgk113@skal8.co
- 성사 여부: 제안서 발송 예정, 성사 여부 미정
🗂️ 최종 State:
{'full_text': ' 안녕하세요 오개광고대 행사 올리질 굿입니다. 혹시 언어부 담당자 맞으신가요? 네 맞습니다.  네 안녕하세요. 다름이 아니라 이번 신제품 출시에 맞춰서 저희가 홍대 지하철 매체를 제한드리고자 하는데 혹시 관심 있으실까요? 홍대가 언어부 주요 타계층인 mg세대와 딱 맞을 것 같아서요.  아 저희가 이번에 홍대는 말고 성수의 파버스토리 계획인데 혹시 성숙 좀 매체도 있나요? 네 물론이죠. 성수역 근처 뿐만 아니라 성수 카페 거리 중심으로 저희가 여러 매체를 보유하고 있는데 혹시 제가

1️⃣ Whisper로 음성 → 텍스트 변환

In [4]:
from faster_whisper import WhisperModel

model = WhisperModel("base")  # 필요하면 small/medium/large 사용 가능
segments, info = model.transcribe("calling_data.m4a")

full_text = " ".join([seg.text for seg in segments])
print("📝 인식된 텍스트:")
print(full_text)

📝 인식된 텍스트:
 안녕하세요 오개광고대 행사 올리질 굿입니다. 혹시 언어부 담당자 맞으신가요? 네 맞습니다.  네 안녕하세요. 다름이 아니라 이번 신제품 출시에 맞춰서 저희가 홍대 지하철 매체를 제한드리고자 하는데 혹시 관심 있으실까요? 홍대가 언어부 주요 타계층인 mg세대와 딱 맞을 것 같아서요.  아 저희가 이번에 홍대는 말고 성수의 파버스토리 계획인데 혹시 성숙 좀 매체도 있나요? 네 물론이죠. 성수역 근처 뿐만 아니라 성수 카페 거리 중심으로 저희가 여러 매체를 보유하고 있는데 혹시 제가 미팅으로 자세하게 설명드려도 될까요? 죄송한데 제가 미팅 어릴 것 같고요. 이멜로 일단 제한서 보내주세요. 네 그럼 저희 통합 오개 광고 제한서와 함께 성수 파벅을 맞춘 제한서 따로 보내드리겠습니다.  혹시 담담자분 성함과 이메일 알 수 있을까요? 제 이름은 이 효정이고요. 이메일은 sgsgk113 골뱅이 스크라다컴입니다. 네 sgsgk113 골뱅이 skal8.co에 맞으시죠? 오늘 중으로 매일 드리겠습니다. 감사합니다. 네 감사합니다.


In [5]:
import openai
import os
from dotenv import load_dotenv

load_dotenv()
client = openai.OpenAI()  # 최신버전: client 객체 생성

def summarize_and_extract(text):
    prompt = f"""
    다음 고객과의 통화 내용을 요약하고, 고객의 요구사항을 다음 필드별로 정리해줘:
    - 지역:
    - 매체:
    - 타겟층:
    - 비고 (미팅 날짜, 추후 연락 등):
    - 연락처:
    - 성사 여부:

    통화 내용:
    \"\"\"{text}\"\"\" 
    """

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "너는 영업 담당자의 어시스턴트야. 고객의 요구사항을 정확하게 정리해야 해."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2
    )

    return response.choices[0].message.content

# full_text 사용
summary = summarize_and_extract(full_text)

print("🔎 요약 및 요구사항 정리:")
print(summary)

🔎 요약 및 요구사항 정리:
- 지역: 성수
- 매체: 성수역 근처 및 성수 카페 거리 중심의 다양한 매체
- 타겟층: MG세대
- 비고 (미팅 날짜, 추후 연락 등): 고객이 미팅 대신 이메일로 제안서를 받기를 원함. 오늘 중으로 이메일 발송 예정.
- 연락처: 이메일 - sgsgk113@skal8.co
- 성사 여부: 제안서 발송 후 결정될 예정 (현재 성사 여부 미확정)


In [ ]:
import requests
import json
import time
import os
from pydub import AudioSegment

# 네이버 클라우드 플랫폼에서 발급받은 키
CLOVA_API_URL = "https://naveropenapi.apigw.ntruss.com/recog/v1/stt"
ACCESS_KEY = ""
SECRET_KEY = ""

def compress_audio(input_file, output_file, format="wav", bitrate="64k"):
    """오디오 파일을 압축하여 크기를 줄입니다."""
    try:
        audio = AudioSegment.from_file(input_file)
        audio.export(output_file, format=format, bitrate=bitrate)
        print(f"✅ 파일 압축 완료: {output_file} (원본 크기의 약 {os.path.getsize(output_file)/os.path.getsize(input_file)*100:.1f}%)")
        return output_file
    except Exception as e:
        print(f"❗ 파일 압축 실패: {str(e)}")
        return None

def clova_speech_to_text(file_path):
    headers = {
        "X-NCP-APIGW-API-KEY-ID": ACCESS_KEY,
        "X-NCP-APIGW-API-KEY": SECRET_KEY,
        "Content-Type": "application/octet-stream"
    }

    params = {
        "lang": "Kor"  # 한국어 코드
    }

    with open(file_path, 'rb') as f:
        audio_data = f.read()

    response = requests.post(CLOVA_API_URL, headers=headers, params=params, data=audio_data)

    if response.status_code == 200:
        result_json = response.json()

        # 응답에서 텍스트 추출
        text = result_json.get('text', '')
        return text

    else:
        print(f"❗ CLOVA API 요청 실패: {response.status_code}")
        print(response.text)
        return None

def process_multiple_audios(file_list):
    """여러 개의 오디오 파일을 변환하고 결과를 합침."""
    full_transcript = ""
    for file_path in file_list:
        # print(f"🔎 변환 중: {file_path}")
        text = clova_speech_to_text(file_path)
        # print(f"✅ 변환 결과: {text}\n")
        full_transcript += text + " "
    return full_transcript.strip()

# ✅ 불러올 오디오 파일들
audio_files = ["calling_data1.wav", "calling_data2.wav"]  # 파일명은 실제 파일에 맞게 수정

transcript = process_multiple_audios(audio_files)

print("\n📝 CLOVA 최종 변환 결과:")
print(transcript)


📝 CLOVA 최종 변환 결과:
안녕하세요 옥외광고 대행사 올리 질 것입니다 혹시 어느 부 담당자 맞으신가요 네 맞습니다 네 안녕하세요 다름이 아니라 이번 신제품 출시에 맞춰서 저희가 홍대 지하철 매체를 대체한 드리고자 하는데 혹시 관심 있으실까요 홍대가 어느 부족의 타겟 층엔 엠지 세대와 딱 맞을 것 같아서요 어서 이거 이번에 홍대는 말고 상세 팝업 스토어 리뉴얼 계획인데 혹시 선수촌 매체도 있나요 네 물론이죠 성수역 근처뿐만 아니라 상수 카페거리 중심으로 저희가 여러 매체를 보유하고 있는데 혹시 제가 미팅으로 자세하게 설명 드려도 될까요 죄송한데 제가 미팅 오를 것 같고요 이멜로 있던 최 한서 보내주세요 네 그럼 저희 통합 옥외 광고 제안서와 함께 성수 팝업을 마친 자연서 따로 보내 드리겠습니다 혹시 자연서 따로 보내 드리겠습니다 혹시 담당자 분 성함과 이메일 알수 있을까요 아 제 이름은 이호정이야 그 이메일은 sg 에스지 케이 일산 골뱅이 닷컴입니다 내 sg 에스지 케이 1213 골뱅이 skl 점 시오에 맞으시죠 오늘중으로 메일 드리겠습니다 감사합니다 감사합니다


2️⃣ GPT로 요약 및 요구사항 정리

In [6]:
import openai
import os
from dotenv import load_dotenv

load_dotenv()
client = openai.OpenAI()  # 최신버전: client 객체 생성

def summarize_and_extract(text):
    prompt = f"""
    다음 고객과의 통화 내용을 요약하고, 고객의 요구사항을 다음 필드별로 정리해줘:
    - 지역:
    - 매체:
    - 타겟층:
    - 비고 (미팅 날짜, 추후 연락 등):
    - 연락처:
    - 성사 여부:

    통화 내용:
    \"\"\"{text}\"\"\" 
    """

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "너는 영업 담당자의 어시스턴트야. 고객의 요구사항을 정확하게 정리해야 해."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2
    )

    return response.choices[0].message.content

# full_text 사용
summary = summarize_and_extract(transcript)

print("🔎 요약 및 요구사항 정리:")
print(summary)

🔎 요약 및 요구사항 정리:
- 지역: 홍대, 성수
- 매체: 홍대 지하철, 성수역 근처 및 상수 카페거리 중심의 옥외광고 매체
- 타겟층: MZ 세대
- 비고 (미팅 날짜, 추후 연락 등): 미팅 대신 이메일로 제안서 요청, 오늘 중으로 이메일 발송 예정
- 연락처: 이메일 - sg1213@skl.co.kr
- 성사 여부: 제안서 이메일 발송 후 결정 예정


In [3]:
from transformers import pipeline

# 음성에서 텍스트로 변환
transcriber = pipeline("automatic-speech-recognition", model="kresnik/wav2vec2-large-xlsr-korean")
audio_file = "calling_data.wav"
result = transcriber(audio_file)
print(result["text"])

config.json:   0%|          | 0.00/2.31k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

Some weights of the model checkpoint at kresnik/wav2vec2-large-xlsr-korean were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_v', 'wav2vec2.encoder.pos_conv_embed.conv.weight_g']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at kresnik/wav2vec2-large-xlsr-korean and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRA

tokenizer_config.json:   0%|          | 0.00/161 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/18.2k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]

양냐 세어 오객 광고  행사 올리지 꾸입니다 혹시 어후부 담당자 마지신가요 내받습니다 난냐세 다름 아니라 이번 신제품 출시에 맞춰서 저희가 홍대 지하철 매체를 제안들이 고자 하는데 혹시 관심 미스실까요 홍대가 온후부 요 타게책인 렌지세대와 딱 마을 것 같았서요  소이가 이번을 홍대는 물고 성술 퍼블스토리열 계획인데 혹시 홍습쪽 매트된날 내 물론이자 상소였 근처뿐만 아니라 선수 카페거리 중심으로 저희가 여러 매체를 보유하고 있는데 혹시 제가 미팅으로 자세하게 설명들어도 될까요  최소원들 제비팅오를 부꾸요 이미로있던 제안소 보내 주세네 그럼 저희 통합 오백 광고 제안소와 함께 상습 파법을 같춘 제안서 따로 보내들리겠습니혹시 당당자본 성한과 이미 알 수 있을까요 제 이름은 이후 정이고요 이미은 메레스케이이 산 골백이 스플래더컴입니시지시지케이일리상 골백이 에스케이에이엘에이 쩜 시오에 마지시자 오늘 종으로 매 들리겠습니다 감사합니다 분산니다
